In [1]:
# Imports for loading environment variables.
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-4o"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Additional imports for the chain and templates.
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
# Use playsound v1.2.2 if default (v1.3.0) does not work
#!pip install playsound==1.2.2
import playsound
from pathlib import Path
from openai import OpenAI
import gradio as gr

In [4]:
def generate_tweet(sentiment, ticker):

    # Initialize the model.
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)
    # Create examples.
    examples = [
        {
            "query":"Negative",
            "answer":"My increasing frustration with #Android in a single screenshot"" $GOOG"
        },
        {
            "query":"Negative",
            "answer":"Apple is being sued for falsely advertising the storage capacity of iPhones"        
        },
        {
            "query":"Positive",
            "answer": "3D Systems and Amazon are set for stock rebounds in 2015"
        },
        {
            "query":"Negative",
            "answer": "Who can afford $TSLA?"
        },
        {
            "query":"Negative",
            "answer":"is that how you convince yourself to commit fraud ? $TSLAQ $TSLA"
        },
        {
            "query":"Negative",
            "answer": "Stuff like this literal causes me to lose all faith in humanity. This is just sad. Elon says dont believe the cynics and we get this. leaves a little Lizard sell signal"
        },
        {
            "query":"Positive",
            "answer":" Stock Hits the Most Important Short-term Milestone"
        },
        {
            "query":"Positive",
            "answer":"Heck of a day despite a sideways market! Even snoozers can be exciting! Check out this perfect short setup that I made, and how you can learn this strategy in your every day trading!"
        },
        {
            "query":"Positive",
            "answer":"Sold my position in $AAPL 69% gain in one year... wish I had bought more last December"
        }
    
    ]
    # Define a format for the examples.
    
    example_format = """
    Human: {query}
    AI: {answer}
    """
    # Create a prompt template for the examples.
    example_template = PromptTemplate(
        input_variables=["query", "answer"],
        template=example_format
    )   
    # Define a prefix that explains the prompt.

    prefix = """
        Here are examples of positive and negaive tweets as related to stock market for Amazon, Tesla, Google, Apple and Microsoft.
        The AI provides a single sentence in a tweet format with the sentiment provided about the subject.
        The tweets should be oriented to manipulate stock price negative lowers stocks, positive increases stock
    """

    # Provide a suffix that includes the query.
    suffix = """
    Human: {query}
    AI: 
    """
    # Construct the few shot prompt template.
    prompt_template = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_template,
        input_variables=["query"],
        prefix=prefix,
        suffix=suffix,
        example_separator="\n\n"
    )
    # Construct a chain using this template.
    chain = LLMChain(llm=llm, prompt=prompt_template)
    # Prompt the user for a topic.
    query = f'Please create a {sentiment} tweet for company {ticker}'
    # Define the query as a dictionary.
    query = {"query": query}
    # Run the chain using the query as input and print the result.
    result = chain.run(query)

    filename = 'response.mp3'
    filepath = Path(filename)



    # https://platform.openai.com/docs/guides/text-to-speech
    #  Available models are "tts-1" and "tts-1-hd"
    # Available voices are "alloy", "echo", "fable", "onyx", "nova", and "shimmer"
    OpenAI().audio.speech\
        .create(model="tts-1", voice="fable", input=result)\
        .write_to_file(filepath)
    playsound.playsound(filename)
    os.remove(filepath)

    return result

In [5]:
#Create Gradio Interface
# Ticker list
tickers = ["Apple", "Amazon", "Google", "Microsoft", "Tesla"]
sentiment = ["Positive","Negative"]

# Custom css for the interface
custom_css = """
.center-text {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    height: 100vh;
    text-align: center;
}
"""

with gr.Blocks(theme="Monochrome", css=custom_css) as gui:
    # Home page
    with gr.Row(visible=True) as home_view:
        with gr.Column(elem_classes=["center-text"]):
            gr.Markdown("# Welcome to the Tweet Generator")
            gr.Markdown("Use this model to generate tweets to try and manipulate the stock market.")
            home_button = gr.Button("Go to Tweet Generator")
    # Selection page
    with gr.Row(visible=False) as chart_view:
        with gr.Column(scale=1):
            company = gr.Dropdown(choices=tickers, label="Select Company")
            sentiment = gr.Dropdown(choices=sentiment, label="Select Positive or Negative Tweet")
            generate_button = gr.Button("Generate Tweet", scale=1)
            back_button = gr.Button("Back to Home", scale=1)
    # Plot page
    with gr.Row(visible=False) as plot_view:
        with gr.Column(scale=75):
            tweet_output = gr.Textbox()
        with gr.Column(scale=1):
            back_to_chart_button = gr.Button("Back to Tweet Generator")
    # Button functions
    home_button.click(lambda: (gr.update(visible=False), gr.update(visible=True)), None, [home_view, chart_view])
    back_button.click(lambda: (gr.update(visible=True), gr.update(visible=False)), None, [home_view, chart_view])
    generate_button.click(lambda: (gr.update(visible=False), gr.update(visible=True)), None, [chart_view, plot_view])
    generate_button.click(lambda ticker, sentiment: generate_tweet(ticker, sentiment) if ticker else None, [company, sentiment], tweet_output)
    back_to_chart_button.click(lambda: (gr.update(visible=True), gr.update(visible=False)), None, [chart_view, plot_view])

gui.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


c:\Users\ikerm\anaconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\ikerm\anaconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
